In [4]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass

In [5]:
#making the connection

host = '127.0.0.1'
user = 'root'
pw = getpass()

con = mysql.connector.connect(host= host, user= user, password= pw)

#checking the connection

con.is_connected()

True

In [6]:
#setting the cursor

cursor = con.cursor()

In [7]:
#connection to sakila

def sakila(query):
    cursor.execute(query)
    result = cursor.fetchall()

    return pd.DataFrame(result, columns= cursor.column_names)

sakila(""" SELECT * FROM sakila.rental""")

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
16039,16045,2005-08-23 22:25:26,772,14,2005-08-25 23:54:26,1,2006-02-15 21:30:53
16040,16046,2005-08-23 22:26:47,4364,74,2005-08-27 18:02:47,2,2006-02-15 21:30:53
16041,16047,2005-08-23 22:42:48,2088,114,2005-08-25 02:48:48,2,2006-02-15 21:30:53
16042,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-15 21:30:53


In [8]:
def rentals_month(month, year, engine):
    query1 = f"""SELECT *
                FROM {str(engine)} r
                WHERE r.return_date LIKE "{str(year)}-{str(month)}%";
    """
    return sakila(query1)

rentals_month('07', '2005', 'sakila.rental')

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,3437,2005-06-21 19:20:17,1798,211,2005-07-01 01:09:17,2,2006-02-15 21:30:53
1,3470,2005-07-05 22:49:24,883,565,2005-07-07 19:36:24,1,2006-02-15 21:30:53
2,3471,2005-07-05 22:51:44,1724,242,2005-07-13 01:38:44,2,2006-02-15 21:30:53
3,3472,2005-07-05 22:56:33,841,37,2005-07-13 17:18:33,2,2006-02-15 21:30:53
4,3473,2005-07-05 22:57:34,2735,60,2005-07-12 23:53:34,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
4183,9410,2005-07-30 20:38:05,1789,22,2005-07-31 19:57:05,2,2006-02-15 21:30:53
4184,9457,2005-07-30 22:23:05,2172,487,2005-07-31 23:07:05,2,2006-02-15 21:30:53
4185,9479,2005-07-30 23:22:09,3461,308,2005-07-31 22:26:09,2,2006-02-15 21:30:53
4186,9489,2005-07-30 23:43:32,4011,17,2005-07-31 20:45:32,2,2006-02-15 21:30:53


In [18]:
def rental_count_month(month, year):
    df = rentals_month(f'{month}', f'{year}', 'sakila.rental')
    compare = df.groupby('customer_id').agg({'return_date': 'count'}).rename(columns={'return_date': f'rental_{month}_{year}'})

    return compare

rental_count_month('06', 2005)


,rental_06_2005
customer_id,
1,9
2,1
3,6
4,6
5,7
...,...
595,3
596,6
597,4


In [20]:
df11 = rental_count_month('06', '2005')
df12 = rental_count_month('07', '2005')

display(df11, df12)

,rental_06_2005
customer_id,
1,9
2,1
3,6
4,6
5,7
...,...
595,3
596,6
597,4


,rental_07_2005
customer_id,
1,10
2,2
3,4
4,2
5,12
...,...
595,10
596,5
597,4


In [30]:
def compare_rentals(month1, month2, year):
    df1 = rental_count_month(f'{month1}', f'{year}')
    df2 = rental_count_month(f'{month2}', f'{year}')

    cal = df2[f'rental_{month2}_{year}'] - df1[f'rental_{month1}_{year}']

    total = df1.merge(right=df2, on= 'customer_id')

    total['compare'] = round(cal)

    return total

In [31]:
compare_rentals('06', '08', '2005')

,rental_06_2005,rental_08_2005,compare
customer_id,,,
1,9,13,4.0
2,1,23,22.0
3,6,16,10.0
4,6,14,8.0
5,7,17,10.0
...,...,...,...
595,3,17,14.0
596,6,14,8.0
597,4,15,11.0
